Import required installs

In [ ]:
pip install selenium
pip install webdriver-manager

Scrape Websites

In [20]:
import json
import openai
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def is_valid_article_link(url):
    # Filter out non-article URLs
    if "beap.gemini.yahoo.com" in url:
        return False
    return True

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)
main_page = "https://finance.yahoo.com/topic/stock-market-news"
driver.get(main_page)

# Scroll to the end of the page to ensure all dynamic content is loaded
# body = driver.find_element(By.TAG_NAME, 'body')
# for _ in range(3):  # Adjust the number for more/less scrolling as needed
#     body.send_keys(Keys.PAGE_DOWN)
#     time.sleep(1)  # Wait for the page to load

articles_data = []

try:
    news_items = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li[class*="js-stream-content"]')))
    for item in reversed(news_items):  # Iterate from bottom to top for stability
        data = {'title': '', 'link': '', 'content': 'Content not available.'} 
        try:
            title_element = item.find_element(By.TAG_NAME, 'h3')
            link = title_element.find_element(By.XPATH, './/ancestor::a').get_attribute('href')
            
            if not is_valid_article_link(link):
                print(f"Skipping non-article link: {link}")
                continue  # Skip non-article links

            data['title'] = title_element.text
            data['link'] = link

            # Open a new tab to avoid losing the list of news_items
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
            driver.get(link)
            try:
                caas_body = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.caas-body')))
                paragraphs = caas_body.find_elements(By.TAG_NAME, 'p')
                article_text = ' '.join([paragraph.text for paragraph in paragraphs if paragraph.text])
                data['content'] = article_text if article_text else "Content not available."
            except (TimeoutException, NoSuchElementException):
                print(f"Content not found or page timed out for: {link}")
            
            articles_data.append(data)
            
            # Close the current tab and switch back to the main page tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"Error processing article: {e}")
            # Ensure to close any opened tab in case of error and switch back
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
finally:
    driver.quit()
    with open('articles_data.json', 'w', encoding='utf-8') as f:
        json.dump(articles_data, f, ensure_ascii=False, indent=4)
    print("Data has been saved to 'articles_data.json'")

Setup the prompts

In [ ]:
persona = "You will response as if you are an optimistic, expert stock market analyst on Twitter. You have a deep understanding of financial markets. Respond in the manner of a catchy, hyped up, and detailed Tweet, with relevant hashtags, if needed. Give a definitive recommendation in the response. Limit your response to 280 characters maximum, and only two hastags maximum."
scraping = "Only us the following articles given in the following context to answer any questions:"
context = "Give me a Tweet"
AIMessages = []
AIMessages.append({"role": "system", "content": persona})
AIMessages.append({"role": "system", "content": scraping})
for elem in articles_data:
    AIMessages.append({"role": "system", "content": elem['content']})
AIMessages.append({"role": "user", "content": context})

Prompt the AI Influencer

In [ ]:
response = openai.chat.completions.create(
        model = 'gpt-4-1106-preview',
        temperature = 1,
        messages=AIMessages,
     )
AITweetString = response.choices[0].message.content
print(AITweetString)